In [43]:
import fiona
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from pathlib import Path

In [44]:
#
# Explore the shapefiles
#

# Load the data
project_dir = Path('/Users/merrelbook/Projects/HealthDataVizGA')
data_dir = project_dir / 'DataVisualAnalytics_Industries-cancer' / 'data_raw'
data_path = data_dir / 'US_Shape_Files' / 'cb_2018_us_county_500k.zip'

# Read direct from the .zip file -- See: https://geopandas.org/io.html
gdf = gpd.read_file('zip://'+str(data_path))
# states = geopandas.read_file(zipfile)

In [61]:
# SOURCE: https://github.com/geopandas/geopandas/issues/834
from shapely import geometry
upcast_dispatch = {geometry.Point: geometry.MultiPoint, 
                   geometry.LineString: geometry.MultiLineString, 
                   geometry.Polygon: geometry.MultiPolygon}

def maybe_cast_to_multigeometry(geom):
    caster = upcast_dispatch.get(type(geom), lambda x: x[0])
    return caster([geom])

In [60]:
georgia_shapes = gdf[gdf.STATEFP=='13']
# georgia_shapes['FIPS'] = georgia_shapes.STATEFP + georgia_shapes.COUNTYFP
# georgia_shapes.head()

georgia_shapes.geometry = georgia_shapes.geometry.apply(maybe_cast_to_multigeometry)
# 

georgia_shapes.to_file(data_path.parent / 'georgia_counties.geojson', driver='GeoJSON')

In [36]:
# Mapbox
token = open(project_dir / ".mapbox_token").read() # you will need your own token